In [33]:
import os
import numpy as np
import glob
import scipy.stats as st

files = glob.glob('*.dat')

for file in files:
    with open(file,'r') as f:
        data = f.read()
        measurements = np.array(data.split('\n'))
        measurements = measurements[measurements != ''].astype(np.float32)
        avg = np.average(measurements)
        std = st.sem(measurements)
        print('{} media {} dp {}'.format(file, avg, std))
        print('intervalo {}'.format(st.t.interval(0.95, len(measurements)-1, loc=avg, scale=st.sem(measurements))))

measurement_m2.dat media 0.1281004548072815 dp 0.000904453918337822
intervalo (0.12630582201065713, 0.12989508760390586)
measurement_m1.dat media 0.08285761624574661 dp 0.0007032399531453848
intervalo (0.08146223560973737, 0.08425299688175586)
